In [10]:
%load_ext autoreload
%autoreload 2
import sys
import os
project_root = os.path.abspath("/Users/joschua/repos/harris_lab/hypnose/hypnose_analysis")
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
import numpy as np 
import utils
from utils import *
from pathlib import Path
import harp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import aeon.io.video as video
from ipywidgets import widgets
from IPython.display import display
import re
import os
import zoneinfo
from datetime import datetime, timezone
from functools import reduce
from src.processing import detect_settings
import src.processing.detect_stage as detect_stage_module

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Loading experiments --> just define the SUBJID and DATE
root = load_experiment(26, 20250730) #can add index for multiple experiments; index=0 as default
stage = detect_stage_module.detect_stage(root)

Loaded experiment: /Volumes/harris/hypnose/rawdata/sub-026_id-077/ses-93_date-20250730/behav/2025-07-30T14-00-21
Found sequence name: QuintupleA_Stage7


In [8]:
data = load_all_streams(root)

Loading data streams from: /Volumes/harris/hypnose/rawdata/sub-026_id-077/ses-93_date-20250730/behav/2025-07-30T14-00-21
Loaded heartbeat data
Calculated real-time offset: 44398 days, 15:43:08
Created timestamp interpolation mapping
Loaded digital_input_data
Loaded output_set
Loaded output_clear
Loaded olfactometer_valves_0
Loaded olfactometer_valves_1
Loaded olfactometer_end_0
Loaded analog_data
Loaded flow_meter
Loaded video_data
Loaded pulse_supply_1
Loaded pulse_supply_2
Created odour_led

Applying time corrections to all data streams...
Applied correction to digital_input_data
Applied correction to output_set
Applied correction to output_clear
Applied correction to olfactometer_valves_0
Applied correction to olfactometer_valves_1
Applied correction to olfactometer_end_0
Applied correction to analog_data
Applied correction to flow_meter
Applied correction to video_data
Applied correction to pulse_supply_1
Applied correction to pulse_supply_2
Applied correction to odour_led
Skipped 

In [13]:
events = load_experiment_events(root)

Loading experiment events...
Loaded heartbeat data for timing synchronization
Calculated real-time offset: 44398 days, 15:43:08
Created timestamp interpolation mapping
Found 4 experiment event files
Processing event file: ExperimentEvents_1904-01-09T00-00-00.csv with 471 rows
Using Seconds column with interpolation
Applied real-time offset: 44398 days, 15:43:08
Found Value column with values: ['EndInitiation' 'Reset' 'ChooseRandomSequence' 'SampleRewardCondition'
 'InitiationSequence' 'AwaitReward']
Found 88 EndInitiation events
Found 88 InitiationSequence events
Found 88 Reset events
Found 31 AwaitReward events
Found 88 SampleRewardCondition events
Found 88 ChooseRandomSequence events
Processing event file: ExperimentEvents_1904-01-08T23-00-00.csv with 546 rows
Using Seconds column with interpolation
Applied real-time offset: 44398 days, 15:43:08
Found Value column with values: ['ChooseRandomSequence' 'SampleRewardCondition' 'InitiationSequence'
 'EndInitiation' 'AwaitReward' 'Reset']

In [16]:
# Get EndInitiation events
end_initiation_df = events['combined_end_initiation_df']

# Print header info
print("=== END INITIATION EVENTS ===")
print(f"Shape: {end_initiation_df.shape}")
print(f"Columns: {end_initiation_df.columns.tolist()}")

# Show first few rows
print("\nFirst 5 events:")
print(end_initiation_df.head())

# Time range
if not end_initiation_df.empty:
    times = end_initiation_df['Time']
    print(f"\nTime range:")
    print(f"First event: {times.iloc[0]}")
    print(f"Last event: {times.iloc[-1]}")
    print(f"Total duration: {times.iloc[-1] - times.iloc[0]}")
    print(f"Total events: {len(times)}")
else:
    print("\nNo EndInitiation events found!")

=== END INITIATION EVENTS ===
Shape: (341, 2)
Columns: ['Time', 'EndInitiation']

First 5 events:
                        Time  EndInitiation
0 2025-07-30 15:01:06.564992           True
1 2025-07-30 15:01:40.204000           True
2 2025-07-30 15:01:46.348992           True
3 2025-07-30 15:01:51.214976           True
4 2025-07-30 15:01:58.142976           True

Time range:
First event: 2025-07-30 15:01:06.564992
Last event: 2025-07-30 17:51:21.785984
Total duration: 0 days 02:50:15.220992
Total events: 341
